# Collective Communication


<table>
    <tr style="background-color:transparent">
        <td><img src="images/collective_comm.gif" width=400/></td>
    </tr>
</table>

[Image Source](https://computing.llnl.gov/tutorials/parallel_comp/images/collective_comm.gif)

# Scatter

Rank 0 acts as a leader, creating a list and scattering it out to all
ranks evenly

```julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing
recv_buf = Vector{Float64}(undef, size)

if rank == 0
    send_buf = collect(1:size) .* 100
    print("Original array on rank 0:\n $(send_buf)\n")
end
    
recv_buf = MPI.Scatter(send_buf, Int, comm; root=0)
print("I got this on rank $(rank):\n $(recv_buf)\n")
```

In [1]:
%%file Scatter1.jl
#!/usr/bin/env julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing
recv_buf = Vector{Float64}(undef, size)

if rank == 0
    send_buf = collect(1:size) .* 100
    print("Original array on rank 0:\n $(send_buf)\n")
end
    
recv_buf = MPI.Scatter(send_buf, Int, comm; root=0)
print("I got this on rank $(rank):\n $(recv_buf)\n")

Writing Scatter1.jl


If the preceding code is saved to a file `Scatter1.jl`, we can run

In [2]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia Scatter1.jl

Original array on rank 0:
 [100, 200]


I got this on rank 0:
 100
I got this on rank 1:
 200


## Scatter!

```julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing
recv_buf = Vector{Float64}(undef, size)

if rank == 0
    send_buf = rand(Float64, (size, size))
    print("Original array on rank 0:\n $(send_buf)\n")
end
    
MPI.Scatter!(send_buf, recv_buf, comm; root=0)
print("I got this array on $(rank):\n $(recv_buf)\n")
```

In [3]:
%%file Scatter2.jl
#!/usr/bin/env julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing
recv_buf = Vector{Float64}(undef, size)

if rank == 0
    send_buf = rand(Float64, (size, size))
    print("Original array on rank 0:\n $(send_buf)\n")
end
    
MPI.Scatter!(send_buf, recv_buf, comm; root=0)
print("I got this array on $(rank):\n $(recv_buf)\n")

Writing Scatter2.jl


If the preceding code is saved to a file `Scatter2.jl`, we can run

In [4]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia Scatter2.jl

Original array on rank 0:
 [0.6185207684136137 0.968588231499006; 0.35492104456879925 0.315139582575782]


I got this array on 0:
 [0.6185207684136137, 0.35492104456879925]


I got this array on 1:
 [0.968588231499006, 0.315139582575782]


## Scatterv!

```julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    recv_buf = Vector{Float64}(undef, size-1)
    send_buf = rand(Float64, (size, size))
    print("Original array on rank 0:\n $(send_buf)\n")
else
    recv_buf = Vector{Float64}(undef, size+1)
end

lengths = [size-1, size+1]
offsets = [0, size-1]
    
MPI.Scatterv!(VBuffer(send_buf, lengths, offsets, MPI.DOUBLE), recv_buf, comm; root=0)
print("I got this array on $(rank):\n $(recv_buf)\n")
```

In [5]:
%%file Scatter3.jl
#!/usr/bin/env julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    recv_buf = Vector{Float64}(undef, size-1)
    send_buf = rand(Float64, (size, size))
    print("Original array on rank 0:\n $(send_buf)\n")
else
    recv_buf = Vector{Float64}(undef, size+1)
end

lengths = [size-1, size+1]
offsets = [0, size-1]
    
MPI.Scatterv!(VBuffer(send_buf, lengths, offsets, MPI.DOUBLE), recv_buf, comm; root=0)
print("I got this array on $(rank):\n $(recv_buf)\n")

Writing Scatter3.jl


If the preceding code is saved to a file `Scatter3.jl`, we can run

In [6]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia Scatter3.jl

Original array on rank 0:
 [0.72533974740034 0.011240995965466904; 0.6883700648760186 0.17922077348645482]


I got this array on 0:
 [0.72533974740034]
I got this array on 1:
 [0.6883700648760186, 0.011240995965466904, 0.17922077348645482]


# Gather

`Gather` is a command that collects results from all processes into a list.

```julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    send_buf = collect(1:size) .* 100
    print("Original array on rank 0:\n $(send_buf)\n" )
end
    
v = MPI.Scatter(send_buf, Int, comm; root=0)
print("I got this on $(rank):\n $(v)\n")

v = v * v

recv_buf = MPI.Gather(v, comm; root=0)
if rank == 0
    print("New array on rank 0:\n $(recv_buf)")
end
```

In [7]:
%%file Gather.jl
#!/usr/bin/env julia
using MPI

MPI.Init()
comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    send_buf = collect(1:size) .* 100
    print("Original array on rank 0:\n $(send_buf)\n" )
end
    
v = MPI.Scatter(send_buf, Int, comm; root=0)
print("I got this on $(rank):\n $(v)\n")

v = v * v

recv_buf = MPI.Gather(v, comm; root=0)
if rank == 0
    print("New array on rank 0:\n $(recv_buf)")
end

Writing Gather.jl


If the preceding code is saved to a file `Gather.jl`, we can run

In [8]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia Gather.jl

Original array on rank 0:
 [100, 200]


I got this on 0:
 100
I got this on 1:
 200


New array on rank 0:
 [10000, 40000]

See also [`Gather!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Gather!), [`Gatherv!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Gatherv!), and [`Allgather`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Allgather)/[`Allgather!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Allgather!)/[`Allgatherv!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Allgatherv!)

# Broadcast

`bcast` sends a single object to every process.

```julia
using MPI

MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    send_buf = "Hello from rank 0!\n"
end

recv_buf = MPI.bcast(send_buf, comm; root=0)
print("Rank $(rank) recieved this message:\n  $(recv_buf)\n")
```

In [9]:
%%file broadcast.jl
#!/usr/bin/env julia
using MPI

MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

send_buf = nothing

if rank == 0
    send_buf = "Hello from rank 0!\n"
end

recv_buf = MPI.bcast(send_buf, comm; root=0)
print("Rank $(rank) recieved this message:\n  $(recv_buf)\n")

Writing broadcast.jl


If the preceding code is saved to a file `broadcast.jl`, we can run

In [10]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia broadcast.jl

Rank 0 recieved this message:
  Hello from rank 0!

Rank 1 recieved this message:
  Hello from rank 0!



See also [`Bcast!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Bcast!).

# Reduce

`Reduce` performs a parallel reduction operation.

```julia
using MPI

MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

v = collect(1:size)
print("Original array on rank $(rank):\n  $(v)\n")
    

recv_buf = MPI.Reduce(v, +, comm; root=0)

if rank == 0
    print("New array on rank 0:\n  $(recv_buf)\n")
    total_sum = sum(recv_buf)
    print("Total sum:\n  $(total_sum)")
end
```

In [11]:
%%file Reduce1.jl
#!/usr/bin/env julia
using MPI

MPI.Init()

comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)
size = MPI.Comm_size(comm)

v = collect(1:size)
print("Original array on rank $(rank):\n  $(v)\n")
    

recv_buf = MPI.Reduce(v, +, comm; root=0)

if rank == 0
    print("New array on rank 0:\n  $(recv_buf)\n")
    total_sum = sum(recv_buf)
    print("Total sum:\n  $(total_sum)")
end

Writing Reduce1.jl


If the preceding code is saved to a file `Reduce1.jl`, we can run

In [12]:
!~/.julia/bin/mpiexecjl --project=.. -np 2 julia Reduce1.jl

Original array on rank 1:
  [1, 2]
Original array on rank 0:
  [1, 2]


New array on rank 0:
  [2, 4]
Total sum:
  6

## Custom reduction operations

```julia
using MPI
MPI.Init()

import Base.+

struct Point{T}
    x::T
    y::T
end

+(A::Point{T}, B::Point{T}) where T = Point{T}(A.x + B.x, A.y + B.y)


comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)

p = Point(rank, rank) 
print("Original Point on rank $(rank):\n  $(p)\n")
    

recv_buf = MPI.Reduce(p, +, comm; root=0)

if rank == 0
    print("\nNew Point on rank 0:\n  $(recv_buf)\n")
end
```

In [13]:
%%file Reduce2.jl
#!/usr/bin/env julia
using MPI
MPI.Init()

import Base.+

struct Point{T}
    x::T
    y::T
end

+(A::Point{T}, B::Point{T}) where T = Point{T}(A.x + B.x, A.y + B.y)


comm = MPI.COMM_WORLD
rank = MPI.Comm_rank(comm)

p = Point(rank, rank) 
print("Original Point on rank $(rank):\n  $(p)\n")
    

recv_buf = MPI.Reduce(p, +, comm; root=0)

if rank == 0
    print("\nNew Point on rank 0:\n  $(recv_buf)\n")
end

Writing Reduce2.jl


If the preceding code is saved to a file `Reduce2.jl`, we can run

In [14]:
!~/.julia/bin/mpiexecjl --project=.. -np 3 julia Reduce2.jl

Original Point on rank 0:
  Point{Int64}(0, 0)


Original Point on rank 1:
  Point{Int64}(1, 1)


Original Point on rank 2:
  Point{Int64}(2, 2)



New Point on rank 0:
  Point{Int64}(3, 3)


See also [`Reduce!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Reduce!) and [`Allreduce`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Allreduce)/[`Allreduce!`](https://juliaparallel.github.io/MPI.jl/latest/collective/#MPI.Allreduce!).

In [15]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>